## Instances initialization

In [1]:
import math
from models import mkp
import random

#items, knapsack, optimum = mkp.open_instance("../instances/chubeas/OR5x100/OR5x100.dat")
# knap = mkp.Knapsack(knapsack[0], items[0])
# print(mkp.Knapsack.pseudo_utilities)
# print(items[0][0].profit)
# print(items[0][0].weight)
# print(knap.pseudo_utility_2())

## SLMS
### Population initialization
Not sure if it's right

In [2]:
def pop_init_random(population_size, list_items, list_constraints):
    """
    Initialize a population of "elite" solutions randomly

    :param population_size: Size of the population
    :type population_size: int
    :param list_items: List of items for the MKP
    :type list_items: list of Item
    :param list_constraints: List of constraints for this knapsack
    :type list_constraints: list of int

    :rtype: list of Knapsack
    :return: The population
    """
    population = []

    for k in range(population_size):
        ks = mkp.Knapsack(list_constraints, list_items)
        for i in range(len(list_items)):
            ks.ks[i] = random.random()
        population += [ks]
    return population

# p = pop_init_random(10, items[0], knapsack[0])
# print(p)

## Generation of initial solution (selection ?)

In [3]:
# Generate a trial solution
def generate_sol(individual, transform_function=math.tanh):
    """
    Generate a trial solution from the moths

    :param individual: The current individual
    :type individual: Knapsack
    :param transform_function: Transform function real to integer
    :type transform_function: typing.Callable

    :rtype: Knapsack
    :return: A trial solution
    """

    individual.ks = [1 if random.uniform(0, 1) < transform_function(x) else 0 for x in individual.ks]
    return individual

# for i in p:
#     generate_sol(i)
#     print(i.ks)

### Repair

In [4]:
def repair(individual):
    """
    Repair the solution given

    :param individual: A solution to repair
    :type individual: Knapsack

    :rtype: None
    :return: Does not return anything, modify the individual given
    """
    resource_consumption = []
    for weight_i in range(len(individual.constraints)):
        resource_consumption += [sum([mkp.Knapsack.items[i].weight[weight_i] for i, val in
                                      enumerate(individual.ks) if val == 1])]

    for i in reversed(mkp.Knapsack.pseudo_utilities):
        # Test if all the constraints are respected
        if False in [res <= const for res, const in zip(resource_consumption, individual.constraints)]:
            if individual.ks[i] == 1:
                individual.ks[i] = 0
                resource_consumption = [res - weight for res, weight in
                                        zip(resource_consumption, mkp.Knapsack.items[i].weight)]
                individual.fitness -= mkp.Knapsack.items[i].profit
        else:
            break

    for i in mkp.Knapsack.pseudo_utilities:
        if individual.ks[i] == 0:
            if not (False in [res + weight <= const for res, const, weight in
                              zip(resource_consumption, individual.constraints, mkp.Knapsack.items[i].weight)]):

                individual.ks[i] = 1
                resource_consumption = [res + weight for res, weight in
                                        zip(resource_consumption, mkp.Knapsack.items[i].weight)]
                individual.fitness += mkp.Knapsack.items[i].profit


### Generation of solution
Lévy flight
x t i + 1 = x ti + α L(s)

α = S max / t 2


L(s) = ( β − 1) Γ ( β − 1) sin(π ( β− 1)/2) / π s β

L(s) represents the step drawn from Lévy flights ??

In [5]:
def calc_step(length, beta):
    """
    Calculate a step s for Lévy flight as seen in
    Nature-Inspired Metaheuristic Algorithms Second Edition - Xin-She Yang

    :param length: Number of items
    :type length: int
    :param beta: The index ?
    :type beta: float

    :rtype: list of int
    :return: The step s for each item
    """
    sigma = (math.gamma(1+beta)*math.sin((math.pi*(beta-1)/2))/
             math.gamma(beta/2)*(beta-1)*2**((beta-2)/2))**(1/(beta-1))

    u = [0]*length
    v = [0]*length
    for i in range(length):
        u[i] = random.random()*sigma
        v[i] = random.random()
    step = [u/abs(v)*(1/(beta-1)) for u, v in zip(u, v)]

    return step

def levyFlight(individuals, scaleFactor, beta=1.5):
    """
    Calculate the Lévy flight for each moth

    :param individuals: A population
    :type individuals: list of mkp.Knapsack
    :param scaleFactor: The scale factor
    :type scaleFactor: float
    :param beta: The index
    :type beta: float

    :rtype: list of mkp.Knapsack
    :return: A list of Knapsack mutated
    """

    s = calc_step(len(mkp.Knapsack.items), beta)
    for i in range(len(individuals)):
        L = ((beta-1)*math.gamma(beta-1)*math.sin(math.pi*(beta-1)/2))\
            /(math.pi*s[i]**beta)
        individuals[i].ks = [item + scaleFactor * L for item in individuals[i].ks]

    return individuals

### Generation of solution
self learning operator </br>
The max step S max = 1 . 0 </br>
acceleration factor φ = 0 . 618 </br>
the index β = 1 . 5. </br>
λ = α = S max / t 2 ??
where scale factor λ is set to a random number drawn by the standard uniform distribution  ?

In [6]:
def sl(individuals, learningTime, scaleFactor, phi, best):
    for i in range(len(individuals)):
        for j in range(learningTime):
            new = mkp.Knapsack(individuals[i].constraints, individuals[i].items)
            if i == 0:
                random_better = random.choice(individuals[:i+1])
            else:
                random_better = random.choice(individuals[:i])
            if random.random() < 0.5:
                new.ks = [scaleFactor * (random_better.ks[item] + phi * (best.ks[item] - individuals[i].ks[item]))
                          for item in range(len(mkp.Knapsack.items))]
            else:
                new.ks = [scaleFactor * (random_better.ks[item] + (1/phi) * (best.ks[item] - individuals[i].ks[item]))
                          for item in range(len(mkp.Knapsack.items))]
            new.fit()
            if new.fitness > individuals[i].fitness:
                individuals[i] = new

### ALGORITHM

In [7]:
def slms(population, maxFit=100000, maxStep=1.0, phi=0.618):
    for individual in population:
        generate_sol(individual)
        repair(individual)
        individual.fit()
    population.sort(key=lambda x: x.fitness, reverse=True)

    i = 0
    generation = 1
    while i < maxFit:
        # Have to change in case population length is odd
        bestInd = population[0]
        pop_1 = population[:int(len(population)/2)]
        pop_2 = population[int(len(population)/2):]

        levyFlight(pop_1, (maxStep/generation**2))
        sl(pop_2, 1, random.random(), phi, bestInd)

        for individual in population:
            generate_sol(individual)
            repair(individual)
            individual.fit()
        population.sort(key=lambda x: x.fitness, reverse=True)

        i += len(population)
        generation += 1
    return population[0]

items, knapsack, optimum = mkp.open_instance("../instances/chubeas/OR5x100/OR5x100.dat")
random.seed(10)
sol = slms(pop_init_random(50, items[0], knapsack[0]))
print(sol.fitness)
print(optimum)

24003
0
